In [26]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

%matplotlib qt

### Load in Annual Surface Temperature Change Data

In [27]:
# Annual Surface Temperature Change Data
AST = pd.read_csv("Annual_Surface_Temperature_Change.csv")


years = np.arange(1961, 2023)[:, np.newaxis]
# World Data
deltaST_world = AST[AST['Country'] == 'World'].loc[:, 'F1961':'F2022'].values.reshape(-1,1)
deltaST_greenland = AST[AST['Country'] == 'Greenland'].loc[:, 'F1961':'F2022'].values.reshape(-1,1)


### Create Polynomial Regression Model

In [28]:
# Create a polynomial regression model

degree = 2
polyreg_model = make_pipeline(PolynomialFeatures(degree), LinearRegression())

### Fit Model to Change in World Surface Temperature Data

In [29]:

polyreg_model.fit(years, deltaST_world)
pred_yrsST = np.linspace(1961, 2023)[:, np.newaxis]
deltaSTW_fit = polyreg_model.predict(pred_yrsST)

In [30]:
plt.clf()

plt.figure(1)
plt.plot(years, deltaST_world)
plt.plot(pred_yrsST, deltaSTW_fit)
plt.title("Change in World Surface Temperature")
plt.xlabel('Year')
plt.ylabel('Change in Surface Temperature $[^oC]$')
plt.grid('minor')
plt.show()


In [31]:
coefficients = polyreg_model.named_steps['linearregression'].coef_.reshape(-1,1)

# Derivative function
def polynomial_derivative(x):
    return sum(coefficients[j] * j * x**(j-1) for j in range(1, len(coefficients)))

# Generate points for evaluation

AST_derivative = np.array([polynomial_derivative(xi) for xi in pred_yrsST])

# Plotting the derivative
plt.figure(2)
plt.plot(pred_yrsST, AST_derivative)
plt.title("Change in Annual Surface Temperatures")
plt.xlabel("Years")
plt.ylabel("Rate of Change [$^oC$ / yr]")
plt.legend()
plt.grid(True)
plt.show()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


### Load in Atmospheric CO2 Concentrations

In [32]:
CO2 = pd.read_csv("Atmospheric_CO₂_Concentrations.csv")
CO2_ppm = CO2[(CO2['Unit'] == 'Parts Per Million') & (CO2['Date'].str.contains('M12'))]['Value'].values.reshape(-1,1)

yrsCO2 = np.arange(1958, 2024)[:, np.newaxis]



### Fit Model to CO2 Concentrations

In [33]:
polyreg_model.fit(yrsCO2, CO2_ppm)
pred_yrsCO2 = np.linspace(1961, 2023)[:, np.newaxis]
CO2_ppm_fit = polyreg_model.predict(pred_yrsCO2)

In [34]:


plt.figure(3)
plt.plot(yrsCO2, CO2_ppm)
plt.plot(pred_yrsCO2, CO2_ppm_fit)
plt.title("CO2 Concentrations")
plt.xlabel('Year')
plt.ylabel('CO2 in Atmosphere [PPM]')
plt.grid('minor')
plt.show()

In [35]:
coefficients = polyreg_model.named_steps['linearregression'].coef_.reshape(-1,1)

# Generate points for evaluation

CO2_derivative = np.array([polynomial_derivative(xi) for xi in pred_yrsCO2])

# Plotting the derivative
plt.figure(4)
plt.plot(pred_yrsCO2, CO2_derivative)
plt.title("Change in CO2 Concentrations")
plt.xlabel("Years")
plt.ylabel("Rate of Change [ppm / yr]")
plt.legend()
plt.grid(True)
plt.show()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


### Correlation Analysis Temperatures and CO2 Concentrations

In [36]:
deltaST_world = list(map(float, deltaST_world))
CO2_ppmList = list(map(float, CO2_ppm[4:]))

df = pd.DataFrame({'temperatures': deltaST_world, 
                'CO2': CO2_ppmList})


# Calculate Pearson correlation
pearson_corr = df.corr(method='pearson')

# Calculate Spearman correlation
spearman_corr = df.corr(method='spearman')

# Calculate Kendall correlation
kendall_corr = df.corr(method='kendall')

# Print the correlation coefficients
print("Pearson correlation coefficient:")
print(pearson_corr)

print("\nSpearman correlation coefficient:")
print(spearman_corr)

print("\nKendall correlation coefficient:")
print(kendall_corr)

Pearson correlation coefficient:
              temperatures       CO2
temperatures      1.000000  0.940941
CO2               0.940941  1.000000

Spearman correlation coefficient:
              temperatures       CO2
temperatures      1.000000  0.926883
CO2               0.926883  1.000000

Kendall correlation coefficient:
              temperatures       CO2
temperatures      1.000000  0.771225
CO2               0.771225  1.000000


C:\Users\damia\AppData\Local\Temp\ipykernel_26120\3548429341.py:1: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  deltaST_world = list(map(float, deltaST_world))
C:\Users\damia\AppData\Local\Temp\ipykernel_26120\3548429341.py:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  CO2_ppmList = list(map(float, CO2_ppm[4:]))


In [37]:
# Create some mock data
fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('Year')
ax1.set_ylabel('Change in Surface Temperature $[^oC]$', color=color)
ax1.plot(pred_yrsST, deltaSTW_fit, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('CO2 in Atmosphere [PPM]', color=color)  # we already handled the x-label with ax1
ax2.plot(pred_yrsST, CO2_ppm_fit, color=color)
ax2.tick_params(axis='y', labelcolor=color)

#fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.title("Regression Lines Overlaid")
plt.show()
